# 0. What is visit history?
* Ｇoal: 分析Chrome瀏覽器的使用歷程檔
* What-is: 瀏覽器的log（即所謂的使用歷程檔）是什麼？上面有什麼資訊？請依照以以下連結的指示，開啟Chrome瀏覽器，找到右上角的工具選項，打開「記錄」。 http://www.googlechrometutorial.com/google-chrome-general-settings/Google-chrome-web-browser-history.html
* Reference
    * [How Google Chrome stores web history](http://lowmanio.co.uk/blog/entries/how-google-chrome-stores-web-history/)
    * http://superuser.com/questions/627253/storing-browser-history-for-personal-analytics
    * http://www.v2ex.com/t/60255

#1. Copy chrome history out
* 如何獲取Chrome History檔案？
* 「access chrome history data」，之後應可找到下列網址 http://stackoverflow.com/questions/2562092/how-to-access-google-chrome-browser-history-programmatically-on-local-machine
解釋】從上面鏈結可獲知Chrome History以sqlite資料庫的檔案形態放在使用者個人電腦端（並非在雲端）

## Windows
* 如果是Windows的話是放在「C:\Documents and Settings\Local Settings\Application Data\Google\Chrome\User Data\Default」檔名為History。
* 該檔案可能是隱藏檔，所以你必須自「資料夾選項」中開啟可看見隱藏檔的選項（以下資訊參考http://www.chromium.org/user-experience/user-data-directory）。
* __Win8__:  C:\Documents and Settings\Local Settings\Application Data\Google\Chrome\User Data\Default
* __Win 7__: C:\Users\[USERNAME]\AppData\Local\Google\Chrome\User Data\Default
* __WinXP__: C:\Documents and Settings\[USERNAME]\Local Settings\Application Data\Google\Chrome\User Data\Default

##  Mac
* 若使用Mac電腦的話，History檔可能會放置在以下兩個位址：
    * /Library（資源庫）/Application Support/Google/Chrome/Default/History
    * /Users/[Your name]/Library（資源庫）/Application Support/Google/Chrome/Default/History
* Mac的History檔為隱藏檔，需要用以下的方法取消隱藏
    * 如果該檔案被隱藏無法搜尋的話，請在spotlight搜尋terminor，會開啟如下視窗（方法來源：view hidden folders on your mac using terminal０）
        * 輸入defaults write com.apple.finder AppleShowAllFiles TRUE 
        * 輸入killall Finder即可找到該資料夾
        * 之後請再輸入defaults write com.apple.finder AppleShowAllFiles FALSE 
        * 再輸入killall Finder即可恢復原狀。
    * <font color = red>或者用這方法：打開Finder後，按住Option，點選上方「前往」中的「前往資料夾」，然後直接貼上上述兩個網址之一，注意要把（資源庫）拿掉，並且把你的使用者名稱貼上。例如：rawdata/Users/jirlong/Library/Application Support/Google/Chrome/Default/History</font>

#2. Browse the History file
* Chrome history是sqlite檔而無法被一般的程式（例如NotePad）開啟。 那麼，何謂sqlite檔案？他和一般資料庫檔案（例如.mdb）有何不同？
* 無論是Firefox或Chrome history均以sqlite的資料庫格式儲存成為一個檔案，所以現在需要用一個sqlite browser來開啟所下載的history檔。
* 請上網搜尋"sqlite reader mac"或"sqlite reader windows"（Mac或Windows視你所用的系統而定，關鍵字reader可改為browser），下載並安裝。http://sourceforge.net/projects/sqlitebrowser/
* 下載該工具並且打開你所下載的History檔案。觀查History的內容，並告訴我裡面記錄了哪些資料，又以何種形式來記錄？

##2.1 Brief introduction to chrome history
* 安裝完畢後用其打開你所下載的History檔案，應可見到如下的界面。共分為三個主要部分，分別為：
    * Database Structure：用以觀看整個資料庫的綱要或欄位
    * Browse Data：用以瀏覽資料庫內存的資料<--我們最主要要看這個資料。
    * Execute SQL：允許你用這個界面操作SQL指令以查詢
    * <img src="Figures/sqlbrowser.png" style="width: 50%; height: 50%"/>
* 點選Browse Data後應可見到以下界面，開啟Table的下拉式選單，一共可發現以下資料庫的資料表：
    * meta
    * downloads：下載的檔案
    * download_url_chains
    * urls：曾經拜訪過的urls的ID與網址對照表
    * visits：主要的瀏覽歷程資料表，其中需要以ID查詢瀏覽時所拜訪的urls（查詢前項urls資料表）
    * visit_source
    * keyword_search_terms：查詢關鍵字
    * segments
    * segment_usage
* 請開啟urls資料表並開啟「檔案>export」的選項將該資料表另存為csv（comma-separated value）檔。再開啟visits資料表，並用上述的方法另存為csv檔。只要儲存為csv檔後，不僅易於程式處理，而且匯入Excel也會十分方便。

##2.2 Tables of chrome history
* History Table – urls. urls資料表主要記錄了你所曾經拜訪過的所有的網頁網址，每一筆記錄就是一筆唯一的網址，如果你曾經拜訪過某網址兩次，他也只會有一筆記錄，它記錄了以下欄位：
    * id：該網址的id，用以索引
    * url：該網址本身
    * title：該網頁的title（可能會從缺）
    * visit_count：使用者在這段時間中一共拜訪該網頁幾次
    * typed_count
    * last_visit_time：使用者最後一次拜訪該網頁的時間（你在看網頁的時候，他有時候會告訴你「您最近一次拜訪時間為...」就是看這個欄位）
    * hidden：是否為隱藏的記錄
    * favicon_id

## 2.3 SQL queries to retrieve data
* 通常類似的SQLite Database Browser都會提供輸入資料庫查詢命令的功能，點選Execute SQL後即可看到輸入SQL查詢命令的欄位。例如我們希望查詢urls這個表中，究竟有哪些網址，並且按照該網址的拜訪次數來排序的話，可以輸入「select visit_count, url from urls order by visit_count」測試之。
* 如果我今天想要查詢資料表visits中的拜訪時間、拜訪的持續時間、拜訪的網址那要怎麼做？此時你若打開資料表visits會發現拜訪網址欄位為url，其內容是一個編號，這個id必須參考對照資料表urls的id和url欄位。因此必須要輸入以下指令：
        SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url
        FROM visits
        INNER JOIN urls
        ON visits.url=urls.id;



#3. Read Chrome history by python
##3.1  import sqlite3
* 為了讀SQLite檔案，必須import sqlite3函式庫，該函式庫為Python安裝時所內建的函式庫，只需import不用另外安裝。
* 把下載下來的History檔案放在同資料中如下圖即可讀取。之後參見https://docs.python.org/2/library/sqlite3.html，
* 該文件的內容說明了sqlite3這個函式庫可以讓使用者建立table、寫入資料並讀取資料庫。

In [1]:
import sqlite3
import urlparse
import datetime

In [2]:
def initDB(dbname):
    conn=sqlite3.connect(dbname)
    cu=conn.cursor()
    return conn, cu

def query(cu, sql):
    cu.execute(sql)
    res=cu.fetchall()
    print "[COMMAND(num_of_res=%d)]:%s"%(len(res), sql)
    return res
def show_table(cu, name="%"):
    sql="SELECT * FROM sqlite_master WHERE type='table' and name like '%s';"%(name)
    return query(cu, sql)

In [3]:
conn, cu = initDB("History")
res = query(cu, "select visit_count, url from urls order by url ;")
for r in res[:10]:
    print "%s\t%s"%(r[0], r[1])
# res = query(cu, "select url,visit_time, from_visit, transition, visit_duration from visits order by id ;") # return top 100 elements
# res = query(cu, "SELECT visits.id, visits.visit_time, visits.visit_duration, urls.url FROM visits INNER JOIN urls ON visits.url=urls.id;")
# tb = show_table(cu) # return all table
# tb = show_table(cu, 'urls') # return table with name like urls

[COMMAND(num_of_res=10341)]:select visit_count, url from urls order by url ;
7	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html
1	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html?uuid=02bb9b6c-134c-4cfc-e186-7875cf78f131
1	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html?uuid=31c77930-ebce-4da8-c1ef-70f981f3ef86
1	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html?uuid=48586a90-9685-4ecb-8f24-c9169ef6f40e
1	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html?uuid=4cd097d7-1426-428a-c5cb-e816c5eb1d1b
1	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html?uuid=5e60507d-4d8b-4ad7-a48a-c5924e246801
1	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html?uuid=86148a94-9dac-4ae3-a613-44ef27224c99
1	chrome-extension://gbkeegbaiigmenfmjfclcdgdpimamgkj/views/app.html?uuid=8fac6ec1-82ca-4c05-ab8d-154130e17852
1	data:text/html,%3Chtml%3E%3Chead%3E%3Cscript%3Ewindow.location.href%3D%27ab

In [4]:
def top_n_url(cu, n=10):
    res_list = []
    url_dict = {}
    res = query(cu, "select visit_count, url from urls order by url ;")
    for item in res:
        urlseg = item[1].split('/')
        try:
            url_dict.setdefault(urlseg[2], 0)
            url_dict[urlseg[2]] += item[0]
        except:
            pass
    res_list = url_dict.items()
    res_list.sort(key=lambda s:s[1], reverse=True)
    for r in res_list[:10]:
        print r
    return res_list

In [5]:
res = top_n_url(cu, 10) # return top 100 elements with default sql command
# res = top_n_url(cu, None) # return all elements

[COMMAND(num_of_res=10341)]:select visit_count, url from urls order by url ;
(u'www.google.com.tw', 3063)
(u'www.dm5.com', 2248)
(u'mail.google.com', 1612)
(u'www.facebook.com', 895)
(u'docs.google.com', 827)
(u'www.appledaily.com.tw', 586)
(u'comic.sfacg.com', 380)
(u'www.iciba.com', 377)
(u'tw.news.yahoo.com', 333)
(u'localhost:8888', 298)


In [6]:
def top_n_url2(cu, n=None, sql="select url,visit_count from urls order by url ;"):
    res = query(cu, sql)
    uniq_res = {}
    for item in res:
#         urlseg = item[0].split('/')
#         print urlseg[2]
        urlhost = urlparse.urlparse(item[0]).netloc   # for getting host from url
        uniq_res.setdefault(urlhost, 0)
        uniq_res[urlhost] += item[1]
    res_list = uniq_res.items()
    res_list.sort(key=lambda x:x[1],reverse=True)
    for r in res_list[:10]:
        print r
    return res_list[:n]

In [7]:
res = top_n_url2(cu, 10)

[COMMAND(num_of_res=10341)]:select url,visit_count from urls order by url ;
(u'www.google.com.tw', 3063)
(u'www.dm5.com', 2248)
(u'mail.google.com', 1612)
(u'www.facebook.com', 895)
(u'docs.google.com', 827)
(u'www.appledaily.com.tw', 586)
(u'comic.sfacg.com', 380)
(u'www.iciba.com', 377)
(u'tw.news.yahoo.com', 333)
(u'localhost:8888', 298)


In [8]:
def getTime(timestamp, pivot="time"):
    newtime = str(timestamp)
    stripped1 = newtime.strip(' (),L')
    ms = int(stripped1)
    delta = datetime.timedelta(microseconds = ms)
    epoch_start = datetime.datetime(1601,1,1) + datetime.timedelta(hours = 8)

    if pivot=="duration":
        return delta
    else:
        return epoch_start + delta

In [9]:
def loaddata(cu):
    print ".......LOADING DATA"
    segment_dict, query_dict = {}, {}
    res = query(cu, "select id, name, url_id from segments;")
    segment_dict[0]="UNKNOWN"
    for r in res:
        segment_dict[r[0]] = r[1]
    res = query(cu, "select url_id, lower_term from keyword_search_terms;")
    for r in res:
        query_dict[r[0]] = r[1]
    return query_dict, segment_dict

In [10]:
query_dict, segment_dict = loaddata(cu)

.......LOADING DATA
[COMMAND(num_of_res=27)]:select id, name, url_id from segments;
[COMMAND(num_of_res=1190)]:select url_id, lower_term from keyword_search_terms;


In [11]:
def get_transition(transition):
    code = 0xff
    transition = code & transition

    flag=""
    if transition==0:
        flag = "CLICK_LINK"     ##"LINK: User reached page by clicking a link on another page"
    elif transition==1:
        flag = "TYPE_URL"       ##"TYPED: User typed page in URL bar"
    elif transition==2:
        flag = "AUTO_BOOKMARK"  ##"AUTO BOOKMARK: User got to this page through a suggestion in the UI"
    elif transition==3:
        flag = "AUTO_SUBFRAME"  ##"AUTO SUBFRAME: Content automatically loaded in a non-toplevel frame"
    elif transition==4:
        flag = "MANU_SUBFRAME"  ##"MANUAL SUBFRAME: Subframe navigation explicitly requested by the user"
    elif transition==5:
        flag = "GENERATED"      ##"GENERATED: User typed page in the URL bar and selected an entry that did not look like a URL"
    elif transition==6:
        flag = "START PAGE"     ##"START PAGE: Page was specified in the command line or is the start page"
    elif transition==7:
        flag = "FORM SUBMIT"    ##"FORM SUBMIT: User filled out values in a form and submitted it"
    elif transition==8:
        flag = "RELOAD"         ##"RELOAD: User reloaded the page"
    elif transition==9:
        flag = "KEYWORD"        ##"KEYWORD: URL generated from a replaceable keyword other than the default search provider"
    elif transition==10:
        flag = "KEYWORD GENERATED"  ##"KEYWORD GENERATED: Visit was generated by a keyword"
    else:
        flag = "UNKNOW"          ##"Unable to understand the transition value. Check, something is horribly wrong here :"
    return flag

In [27]:
def getvisit(cu, query_dict):
    res=query(cu, "SELECT urls.url, visits.visit_time, visits.visit_duration, visits.id, visits.from_visit, visits.transition, visits.segment_id, urls.id FROM visits INNER JOIN urls ON visits.url=urls.id;")
    parsed_res = []
    for item in res:
        urlhost = urlparse.urlparse(item[0]).netloc
        if item[7] in query_dict:
            queryterm = query_dict[item[7]]
        else:
            queryterm = ""
        parsed_res.append([urlhost, getTime(item[1]), getTime(item[2], "duration"), item[3], item[4], get_transition(item[5]), queryterm])
#         parsed_res.append([urlhost, item[1], item[2]])
    parsed_res.sort(key=lambda x:x[1])
    return parsed_res

In [32]:
visits = getvisit(cu, query_dict)
for v in visits:
    print "%s\t%s\t%s\t%s\t%s\t%s\t%s"%(v[1].strftime('%Y-%m-%d %H:%M:%S'), 
                            v[2].seconds,
                            v[3], 
                            v[4],
                            v[5],
                            v[6],
                            v[0])

[COMMAND(num_of_res=16759)]:SELECT urls.url, visits.visit_time, visits.visit_duration, visits.id, visits.from_visit, visits.transition, visits.segment_id, urls.id FROM visits INNER JOIN urls ON visits.url=urls.id;
2014-12-11 18:27:33	0	9461	0	START PAGE		
2014-12-11 18:27:33	0	9462	9461	CLICK_LINK		
2014-12-11 18:27:38	0	9463	0	FORM SUBMIT		www.dropbox.com
2014-12-11 18:28:35	0	9466	0	AUTO_BOOKMARK		translate.google.com
2014-12-11 18:28:43	0	9468	0	TYPE_URL		mail.google.com
2014-12-11 18:29:57	0	9470	0	CLICK_LINK		mail.google.com
2014-12-11 18:31:06	0	9472	0	CLICK_LINK		mail.google.com
2014-12-11 18:31:15	0	9474	0	CLICK_LINK		mail.google.com
2014-12-11 18:32:00	0	9477	0	CLICK_LINK		mail.google.com
2014-12-11 18:32:36	0	9479	0	CLICK_LINK		mail.google.com
2014-12-11 18:33:18	0	9482	0	CLICK_LINK		www.facebook.com
2014-12-11 18:33:19	0	9483	9482	CLICK_LINK		www.facebook.com
2014-12-11 18:33:20	0	9484	9483	CLICK_LINK		www.facebook.com
2014-12-11 18:33:26	22	9486	9484	CLICK_LINK		www.faceboo

In [19]:
def visitfilter(inlist, FILTER=None, DEL=False):
    res = []
    filter_in = ['dropbox', 'dictionary', 'facebook', 'mail', 'youtube', 'toasty', 'comic', 'dm5', 'mobile01']
    filter_out = ['account', 'ftp', 'itc']
    if FILTER == None:
        return inlist
    if FILTER == "Exclude":
        for item in inlist:
            if len([x for x in filter_out if x in item[0]])==0:
                res.append(item)
    if FILTER == "Include":
        for item in inlist:
            if len([x for x in filter_in if x in item[0]])>0:
                res.append(item)
    return res

In [20]:
def merge_visit(visits):
    print "VISITS BEFORE MERGE:%d"%(len(visits))
    res = []
    uniq_url = None
    for visit in visits:
        if uniq_url != visit[0]:
            uniq_url = visit[0]
            res.append([uniq_url, visit[1], visit[1]+visit[2], visit[2]])
        else:
            try:
                res[-1][2] = visit[1]+visit[2]
                res[-1][3] += visit[2]
            except:
                print res[-1]
    print "VISITS BEFORE MERGE:%d"%(len(res))
    return res

In [23]:

filtered_visits = visitfilter(visits, None) ##FILTER = Exclude, Include, or None
print "%s\t%-14s\t%s\t%-10s\t%-8s\t%s\t%s"%('date', 'time', 'duration', 'from', 'transition', 'to', 'url')
for r in filtered_visits[:10]:
    print "%s\t%-14s\t%s\t%-10s\t%-8s\t%s\t%s"%(r[1], r[2],  r[4], r[5], r[6], r[3], r[0])

date	time          	duration	from      	transition	to	url
2014-12-11 18:27:33.611806	0:00:00       	0	START PAGE	        	9461	
2014-12-11 18:27:33.648313	0:00:00       	9461	CLICK_LINK	        	9462	
2014-12-11 18:27:38.492171	0:00:00       	0	FORM SUBMIT	        	9463	www.dropbox.com
2014-12-11 18:28:35.539529	0:00:00       	0	AUTO_BOOKMARK	        	9466	translate.google.com
2014-12-11 18:28:43.328261	0:00:00       	0	TYPE_URL  	        	9468	mail.google.com
2014-12-11 18:29:57.908242	0:00:00       	0	CLICK_LINK	        	9470	mail.google.com
2014-12-11 18:31:06.427815	0:00:00       	0	CLICK_LINK	        	9472	mail.google.com
2014-12-11 18:31:15.019580	0:00:00       	0	CLICK_LINK	        	9474	mail.google.com
2014-12-11 18:32:00.081421	0:00:00       	0	CLICK_LINK	        	9477	mail.google.com
2014-12-11 18:32:36.042567	0:00:00       	0	CLICK_LINK	        	9479	mail.google.com


In [40]:
merged_visits = merge_visit(filtered_visits)
print merged_visits[0]
for r in merged_visits:
#     print "%s\t%s\t%s\t%s"%(r[0], r[1].isoformat(), r[2].isoformat(), r[3].seconds)
    print "%s\t%s\t%s\t%s"%(r[1].strftime('%Y-%m-%d %H:%M:%S'), r[2].strftime('%Y-%m-%d %H:%M:%S'), r[3].seconds, r[0])

VISITS BEFORE MERGE:16759
VISITS BEFORE MERGE:4790
[u'', datetime.datetime(2014, 12, 11, 18, 27, 33, 611806), datetime.datetime(2014, 12, 11, 18, 27, 33, 648313), datetime.timedelta(0)]
2014-12-11 18:27:33	2014-12-11 18:27:33	0	
2014-12-11 18:27:38	2014-12-11 18:27:38	0	www.dropbox.com
2014-12-11 18:28:35	2014-12-11 18:28:35	0	translate.google.com
2014-12-11 18:28:43	2014-12-11 18:32:36	0	mail.google.com
2014-12-11 18:33:18	2014-12-11 18:33:48	22	www.facebook.com
2014-12-11 18:33:54	2014-12-11 18:35:00	5	mail.google.com
2014-12-11 18:40:53	2014-12-11 18:41:11	0	www.google.com.tw
2014-12-11 18:42:37	2014-12-11 18:43:08	31	www.apple.com
2014-12-11 18:43:51	2014-12-11 18:47:36	147	www.google.com.tw
2014-12-11 18:47:38	2014-12-11 18:48:29	51	pixlr.com
2014-12-11 18:58:36	2014-12-11 18:58:36	0	dropbox.com
2014-12-11 18:59:00	2014-12-11 19:02:32	211	www.dropbox.com
2014-12-11 19:01:52	2014-12-11 19:01:52	0	mail.google.com
2014-12-11 19:04:49	2014-12-11 19:04:49	0	support.apple.com
2014-12-11

In [41]:
def report(vlist, fname):
    fout=file(fname, 'w')
    fout.write('''
<script type="text/javascript" src="https://www.google.com/jsapi?autoload={'modules':[{'name':'visualization',
       'version':'1','packages':['timeline']}]}"></script>
<script type="text/javascript">

google.setOnLoadCallback(drawChart);
function drawChart() {

  var container = document.getElementById('example5.1');
  var chart = new google.visualization.Timeline(container);
  var dataTable = new google.visualization.DataTable();
  dataTable.addColumn({ type: 'string', id: 'Room' });
  dataTable.addColumn({ type: 'string', id: 'Name' });
  dataTable.addColumn({ type: 'date', id: 'Start' });
  dataTable.addColumn({ type: 'date', id: 'End' });
  dataTable.addRows([
    ''')
    for v in vlist:
        fout.write('["%s", "%s", new Date(0,0,0,%d,%d,%d), new Date(0,0,0,%d,%d,%d)],\n'%(v[1].strftime('%Y-%m-%d'), v[0], v[1].hour, v[1].minute, v[1].second, v[2].hour, v[2].minute, v[2].second))
    fout.write('''
        ]);
  var options = {
      colors:['#33ccff', '#ff66cc', '#339933','#ffcc33','#ff0000', '#333333', '#996600'],
    timeline: { rowLabelStyle: {fontName: 'Helvetica', fontSize: 24, color: '#603913' },
                barLabelStyle: { fontName: 'Garamond', fontSize: 14 } }
  };
  chart.draw(dataTable, options);
}
</script>
<div id="example5.1" style="width: 3900px; height: 1000px;"></div>
    ''')
    fout.close()


In [84]:
def getPeriod(vlist, s, e):
    startTime = datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S")
    endTime = datetime.datetime.strptime(e, "%Y-%m-%d %H:%M:%S")
    rlist=[]
    for v in vlist:
        if startTime < v[1] < endTime:
            if v[1].day != v[2].day:
                dayend = datetime.datetime.strptime(v[1].strftime('%Y-%m-%d')+" 23:59:59", "%Y-%m-%d %H:%M:%S")
                nextday = datetime.datetime.strptime(v[2].strftime('%Y-%m-%d')+" 00:00:00", "%Y-%m-%d %H:%M:%S")
#                 rlist.append([v[0], v[1], dayend, dayend-v[1]])
#                 rlist.append([v[0], nextday, v[2], v[2]-nextday])
            else:
                rlist.append(v)
    return rlist
#         if v[1] > v[2]:
#             print v[1].day, v[2].day, v[1].strftime('%Y-%m-%d %H:%M:%S'), v[2].strftime('%Y-%m-%d %H:%M:%S')
#         else:


In [85]:
rlist = getPeriod(merged_visits, "2013-12-01 00:00:00", "2015-01-01 00:00:00")
for r in rlist:
    print r
report(rlist, "viz.html")

[u'', datetime.datetime(2014, 12, 11, 18, 27, 33, 611806), datetime.datetime(2014, 12, 11, 18, 27, 33, 648313), datetime.timedelta(0)]
[u'www.dropbox.com', datetime.datetime(2014, 12, 11, 18, 27, 38, 492171), datetime.datetime(2014, 12, 11, 18, 27, 38, 492171), datetime.timedelta(0)]
[u'translate.google.com', datetime.datetime(2014, 12, 11, 18, 28, 35, 539529), datetime.datetime(2014, 12, 11, 18, 28, 35, 539529), datetime.timedelta(0)]
[u'mail.google.com', datetime.datetime(2014, 12, 11, 18, 28, 43, 328261), datetime.datetime(2014, 12, 11, 18, 32, 36, 42567), datetime.timedelta(0)]
[u'www.facebook.com', datetime.datetime(2014, 12, 11, 18, 33, 18, 983979), datetime.datetime(2014, 12, 11, 18, 33, 48, 266772), datetime.timedelta(0, 22, 213925)]
[u'mail.google.com', datetime.datetime(2014, 12, 11, 18, 33, 54, 781454), datetime.datetime(2014, 12, 11, 18, 35, 0, 998320), datetime.timedelta(0, 5, 765420)]
[u'www.google.com.tw', datetime.datetime(2014, 12, 11, 18, 40, 53, 315652), datetime.dat